Take daily means and good structuring from some files. Goes together with readme file.

In [36]:
import numpy as np
import os
import glob
import pandas as pd

file_savedir = '/home/ccorbella/scratch2_symboliclink/files/station_timeseries_preprocessed/'
file_readdir = '/home/ccorbella/scratch2_symboliclink/files/station_timeseries_orig/'

# Define a function to categorize hours into morning, midday, and evening
def tod(hour):
    if hour < 10:
        return "morning"
    elif 10 <= hour < 16:
        return "midday"
    else:
        return "evening"
    
def make_date_col(df):
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']], errors='coerce')
    return df

# Define weights for each hour
default_weights = {"morning": 0.3, "midday": 0.5, "evening": 0.2}

# Function to dynamically adjust weights based on available time categories
def adjust_weights_ta(group):
    available_times = group["Time_of_Day"].unique()
    num_available = len(available_times)
    
    if num_available == 3:
        weights = default_weights  # Use standard weights
    elif num_available == 2:
        weights = {t: 0.5 for t in available_times}  # Split equally between two
    elif num_available == 1:
        weights = {t: 1.0 for t in available_times}  # Assign 100% weight to the only available
    else:
        return None  # If there's no valid data, return None

    # Apply the adjusted weights
    return (group["Term"] * group["Time_of_Day"].map(weights)).sum() / group["Time_of_Day"].map(weights).sum()



# València

In [39]:
valencia_raw = pd.read_csv(f'{file_readdir}Valencia/Valencia_concatenated.csv',
                           dtype={'Day': 'Int64', 'Month': 'Int64', 'Year': 'Int64', 'Hour': 'Int64'})

valencia_raw = valencia_raw.iloc[:,:9]
valencia_raw = valencia_raw.dropna(subset=['Day', 'Month','Year','Hour'])
valencia_raw = valencia_raw.dropna(subset=['Term', 'Bar(p)', 'Bar(l)'])

valencia_raw["Hour"] = pd.to_numeric(valencia_raw["Hour"], errors='coerce')
valencia_raw["Date"] = pd.to_datetime(valencia_raw["Date"])

valencia_raw["Time_of_Day"] = valencia_raw["Hour"].apply(tod)
valencia_raw["Term"] = pd.to_numeric(valencia_raw['Term'], errors='coerce')
valencia_raw["Bar(p)"] = pd.to_numeric(valencia_raw['Term'], errors='coerce')
valencia_raw["Bar(l)"] = pd.to_numeric(valencia_raw['Term'], errors='coerce')
valencia_raw["Higro1"] = pd.to_numeric(valencia_raw['Term'], errors='coerce')

valencia_raw

,Day,Month,Year,Hour,Term,Bar(p),Bar(l),Higro1,Date,Time_of_Day
0,3,1,1804,7,10.00,10.00,10.00,10.00,1804-01-03,morning
1,3,1,1804,13,10.50,10.50,10.50,10.50,1804-01-03,midday
2,3,1,1804,18,10.25,10.25,10.25,10.25,1804-01-03,evening
3,4,1,1804,7,9.50,9.50,9.50,9.50,1804-01-04,morning
4,4,1,1804,13,9.00,9.00,9.00,9.00,1804-01-04,midday
...,...,...,...,...,...,...,...,...,...,...
59780,30,12,1859,12,14.00,14.00,14.00,14.00,1859-12-30,midday
59781,30,12,1859,18,12.00,12.00,12.00,12.00,1859-12-30,evening
59782,31,12,1859,7,10.00,10.00,10.00,10.00,1859-12-31,morning
59783,31,12,1859,12,14.00,14.00,14.00,14.00,1859-12-31,midday


### Temperature

In [33]:
# Compute weighted averages for each day
valencia = valencia_raw.groupby("Date", group_keys=False).apply(
    lambda group: pd.Series({
        'daymean_ta': adjust_weights_ta(group),
    })
).reset_index()

valencia.to_csv(f'{file_savedir}Valencia_daymean_TMP2m.csv')

/tmp/ipykernel_236428/37156510.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  valencia = valencia_raw.groupby("Date", group_keys=False).apply(


In [32]:
obs_per_day = valencia_raw.groupby('Date').size().reset_index(name='obs_per_day')
obs_per_day[obs_per_day['obs_per_day']<3].shape[0]/valencia.shape[0]

0.0023396373562097877

### Pressure

In [46]:
valencia_p = valencia_raw.groupby("Date").agg(
    Bar_p=("Bar(p)", "mean"),
    Bar_l=("Bar(l)", "mean")
).reset_index()

valencia_p.to_csv(f'{file_savedir}Valencia_daymean_PRMSL.csv')

# Ylitornio

In [74]:
yli = pd.read_csv(f'{file_readdir}CLIM_Data_Ylitornio_1800_1838.csv',
                  delimiter=";", skiprows=6, encoding="Windows-1252")

yli = yli.iloc[:,:7]

try: yli = yli.drop(columns={'Unnamed: 3', 'Observation hour'})
except: print('already cropped df')

yli = make_date_col(yli)

In [82]:
# check for typos in dates
yli[yli.duplicated(subset=['Date'], keep=False)]

,Year,Month,Day,Air pressure,Temperature,Date
66,1800,12,6,25.60,-1.0,1800-12-06
3147,1809,12,6,25.48,-4.0,1800-12-06
3512,1810,12,6,25.40,-10.0,1810-12-06
6799,1810,12,6,25.48,-10.0,1810-12-06
8960,1826,11,5,25.55,-6.7,1826-11-05
9325,1826,11,5,NaN,-2.0,1826-11-05


In [94]:
# ammend the typos and redo date column
yli.at[3147,'Year']=1809
yli.at[6799, 'Year']=1819
yli.at[8960, 'Year']=1825

yli = make_date_col(yli)

# split into Pressure and Temperature
yli_p = yli[['Year', 'Month', 'Day', 'Air pressure']]
yli_p.rename(columns={'Air pressure': 'Pressure'})

yli_ta = yli[['Year', 'Month', 'Day', 'Temperature']]

# save
yli_p.to_csv(f'{file_savedir}Ylitornio_PRMSL.csv')
yli_ta.to_csv(f'{file_savedir}Ylitornio_TMP2m.csv')


## Cádiz

We have 29244-13=29231 days with values in Peter's data from IMPROVE. Now let's count how many we have from the raw. We have no duplicates in dataset, and we have 29231 non-NaNs. Yay! it's the same. It looks like `CSF-TP801-819.txt` is just a small subset of `CSF-TP786-879.txt`.

In [148]:
cadiz1 = pd.read_csv(f'{file_readdir}CSF-TP786-879.txt', sep="\t", header=None)
cadiz2 = pd.read_csv(f'{file_readdir}CSF-TP801-819.txt', sep="\t", header=None)

cadiz1.columns = ['Date', 'Tmin', 'Tmax', 'ta', 'p']
cadiz2.columns = cadiz1.columns

cadiz1 = cadiz1.replace(-999.0, np.NaN)
cadiz2 = cadiz2.replace(-999.0, np.NaN)

# drop if all NaNs
cadiz1 = cadiz1.dropna(subset=['ta', 'p'], how='all')
cadiz2 = cadiz2.dropna(subset=['ta', 'p'], how='all')

cadiz1['Date'] = pd.to_datetime(cadiz1['Date'], format="%d/%m/%Y")
cadiz2['Date'] = pd.to_datetime(cadiz1['Date'], format="%d/%m/%Y")

cadiz1_ta = cadiz1[['Date', 'ta']]
cadiz1_p  = cadiz1[['Date', 'p']]

cadiz1_ta.to_csv(f'{file_savedir}Cadiz_TMP2m.csv')
cadiz1_p.to_csv(f'{file_savedir}Cadiz_PRMSL.csv')

## Stockholm
We already had temperature from `PALAEO-RA_IMPROVE_Milan_17630101-18621231_ta`, which is the same as the third column of the raw file, and now I add pressure.

In [38]:
milan = pd.read_csv(f'{file_readdir}MI_TP763_862.txt', sep=r'\s+', header=None)

milan.columns = ['Date', 'Tmin', 'Tmax', 'ta', 'p']

milan = milan.replace(-999.0, np.NaN)

# drop if all NaNs
milan = milan.dropna(subset=['ta', 'p'], how='all')

milan['Date'] = pd.to_datetime(milan['Date'], format="%d/%m/%Y")

milan_p  = milan[['Date', 'p']]

milan_p.to_csv(f'{file_savedir}Milan_PRMSL.csv')

## Padova
Ja teníem pressure i temperature fins a 1809, ara s'ha d'allargar i per no tenir 2 series de la mateixa station les concatenejo i en faig una de sola, que la utilitzaré per reemplaçar la de la carpeta USB stick Peter.

In [21]:
padova1 = pd.read_csv(f'{file_readdir}PD_PT766_809.txt', sep=r'\s+', header=None)
padova2 = pd.read_csv(f'{file_readdir}PD_PT810_853.txt', sep=r'\s+', header=None)

padova1.columns = ['Date', 'Tmin', 'Tmax', 'ta', 'p']
padova2.columns = padova1.columns

padova1 = padova1.replace(-999.0, np.NaN)
padova2 = padova2.replace(-999.0, np.NaN)

# drop if all NaNs
padova1 = padova1.dropna(subset=['ta', 'p'], how='all')
padova2 = padova2.dropna(subset=['ta', 'p'], how='all')

padova1['Date'] = pd.to_datetime(padova1['Date'], format="%d/%m/%Y")
padova2['Date'] = pd.to_datetime(padova2['Date'], format="%d/%m/%Y")

padova = pd.concat([padova1, padova2], ignore_index=True)

padova_ta = padova[['Date', 'ta']]
padova_p  = padova[['Date', 'p']]

padova_ta.to_csv(f'{file_savedir}Cadiz_TMP2m.csv')
padova_p.to_csv(f'{file_savedir}Cadiz_PRMSL.csv')

## Stockholm
Same as Padova, without need for concatenating files.

In [42]:
stockholm = pd.read_csv(f'{file_readdir}ST_TP756_880.txt', sep=r'\s+', header=None)

stockholm.columns = ['Date', 'ta', 'p']

stockholm = stockholm.replace(-999.0, np.NaN)

# drop if all NaNs
stockholm = stockholm.dropna(subset=['ta', 'p'], how='all')

stockholm['Date'] = pd.to_datetime(stockholm['Date'], format="%d/%m/%Y")

stockholm_p  = stockholm[['Date', 'p']]
stockholm_ta  = stockholm[['Date', 'ta']]

stockholm_p.to_csv(f'{file_savedir}Stockholm_PRMSL.csv')
stockholm_ta.to_csv(f'{file_savedir}Stockholm_TMP2m.csv')

## Uppsala

In [43]:
uppsala = pd.read_csv(f'{file_readdir}UPP_TP722_870.txt', sep=r'\s+', header=None)

uppsala.columns = ['Date', 'ta', 'unknown_A', 'p', 'unknown_B']

uppsala = uppsala.replace(-999.0, np.NaN)

# drop if all NaNs
uppsala = uppsala.dropna(subset=['ta', 'p'], how='all')

uppsala['Date'] = pd.to_datetime(uppsala['Date'], format="%d/%m/%Y")

uppsala_p  = uppsala[['Date', 'p']]
uppsala_ta = uppsala[['Date', 'ta']]

uppsala_p.to_csv(f'{file_savedir}Uppsala_PRMSL.csv')
uppsala_ta.to_csv(f'{file_savedir}Uppsala_TMP2m.csv')

## Bologna


In [38]:
bologna = pd.read_csv(f'{file_readdir}TG_SOUID100862.txt', sep=',', header=13)
bologna = bologna.replace(-9999, np.NaN)
bologna = bologna.drop(columns=' SOUID')
bologna.rename({'    DATE': 'Date', '   TG': 'TG', ' Q_TG':'Q_TG'}, inplace=True, axis='columns')
bologna['Date'] = pd.to_datetime(bologna['Date'], format='ISO8601')
bologna.loc[bologna['TG'].notna() & bologna['Q_TG']==1] # check how many suspect cells (Q_TG==1) and how many missing cells (Q_TG==9) are not empty
bologna

,Date,TG,Q_TG
0,1814-01-01,36.0,0
1,1814-01-02,24.0,0
2,1814-01-03,9.0,0
3,1814-01-04,7.0,0
4,1814-01-05,14.0,0
...,...,...,...
73164,2014-04-26,NaN,9
73165,2014-04-27,NaN,9
73166,2014-04-28,NaN,9
73167,2014-04-29,NaN,9


## Prague

In [3]:
prague = pd.read_csv(f'{file_readdir}TG_STAID000027.txt', sep=',', header=14)
prague = prague.replace(-9999, np.NaN)
prague = prague.drop(columns=' SOUID')
prague.rename({'    DATE': 'Date', '   TG': 'TG', ' Q_TG':'Q_TG'}, inplace=True, axis='columns')
prague['Date'] = pd.to_datetime(prague['Date'], format='ISO8601')
prague.loc[prague['TG'].notna() & prague['Q_TG']==1] # check how many suspect cells (Q_TG==1) and how many missing cells (Q_TG==9) are not empty

,Date,TG,Q_TG
3711,1785-02-28,-213.0,1
30641,1858-11-23,-150.0,1
87607,2014-11-11,120.0,1
87608,2014-11-12,120.0,1
87609,2014-11-13,120.0,1
87610,2014-11-14,120.0,1
87611,2014-11-15,120.0,1


## Paris

In [16]:
paris = pd.read_csv('/home/ccorbella/scratch2_symboliclink/files/station_timeseries_orig/Paris_4_17850101-18720615_mslp.tsv',sep='\t',header=12)
make_date_col(paris)
paris_p = paris[['Year', 'Month', 'Day', 'Value', 'Date']]
paris_p.rename(columns={'Value':'PRMSL'})
paris_p.to_csv(f'{file_savedir}paris_PRMSL.csv')

## London


In [34]:
londonA = pd.read_csv(f'{file_readdir}/London_11_17870101-18221231_mslp.tsv',sep='\t',header=12)
londonB = pd.read_csv(f'{file_readdir}/London_12_18230101-18411231_mslp.tsv',sep='\t',header=12)

london = pd.concat([londonA,londonB], ignore_index=True)
make_date_col(london)

london = london.groupby("Date").agg(
    PRMSL=("Value", "mean"),
).reset_index()
london.to_csv(f'{file_savedir}London_PRMSL.csv')

## Zwanenburg

In [43]:
df = pd.read_csv(f'{file_readdir}DeBilt_1_17430101-18501231_mslp.tsv', sep='\t', header=12)
make_date_col(df)

df = df.groupby('Date').agg(PRMSL=('Value', 'mean')).reset_index()
df.to_csv(f'{file_savedir}Zwanenburg_PRMSL.csv')

## Torino

In [64]:
df_ta = pd.read_csv(f'{file_readdir}torino_tmin-tmax.txt',sep='\t', header=None)
df_ta.columns=['Month', 'Day', 'Year', 'tmin','tmax']
make_date_col(df_ta)
df_ta['TMP2m'] = (df_ta['tmin'] + df_ta['tmax'])/2
df_ta[['Date', 'TMP2m']].to_csv(f'{file_savedir}Torino_TMP2m.csv')

,Month,Day,Year,tmin,tmax,Date,TMP2m
0,1,1,1753,-7.0,-2.4,1753-01-01,-4.70
1,2,1,1753,-4.1,2.5,1753-02-01,-0.80
2,3,1,1753,-5.4,3.0,1753-03-01,-1.20
3,4,1,1753,-0.6,2.5,1753-04-01,0.95
4,5,1,1753,-0.8,3.9,1753-05-01,1.55
...,...,...,...,...,...,...,...
93862,27,12,2009,1.4,9.8,NaT,5.60
93863,28,12,2009,0.3,6.1,NaT,3.20
93864,29,12,2009,-0.7,7.3,NaT,3.30
93865,30,12,2009,0.7,6.5,NaT,3.60


In [65]:
df_p = pd.read_csv(f'{file_readdir}TorinoPR792_953.csv', sep='\t')
df_p.columns=['Year','Month','Day','Hour','PRMSL']
make_date_col(df_p)
df_p[['PRMSL','Date']].to_csv(f'{file_savedir}Torino_PRMSL.csv')